In [ ]:
import pandas as pd
df = pd.read_csv("ConjuntoValidacion.csv")
df

### Pretratamiento

In [ ]:
df.columns = df.columns.str.replace("á","a")
df.columns = df.columns.str.replace("é","e")
df.columns = df.columns.str.replace("í","i")
df.columns = df.columns.str.replace("ó","o")
df.columns = df.columns.str.replace("ú","u")
df.columns = df.columns.str.replace("ñ","n")
df.columns = df.columns.str.replace("(","")
df.columns = df.columns.str.replace(")","")
df.columns = df.columns.str.replace('/ mL',"")
df.columns = df.columns.str.replace("/mL","")
df.columns = df.columns.str.replace("g/dL","")
df.columns = df.columns.str.replace(" %","")
df.columns = df.columns.str.upper()
df.columns = df.columns.str.strip()
df.columns = df.columns.str.replace("º ","_")
df.columns = df.columns.str.replace("º","_")
df.columns = df.columns.str.replace(" ","_")
df.columns = df.columns.str.replace("/","_")

df.columns

In [ ]:
# cambiar TRT_6MESES_SIN_1_MES por TRT
df = df.rename(columns={'TRT_6MESES_SIN_1_MES': 'TRT'})
df = df.rename(columns={'N_PASTILLAS': 'N_PASTILLAS_D_INICIO'})

In [ ]:
df.drop(["CRITERIO_DE_GRUPO","NHC"],inplace=True,axis=1)

In [ ]:
df.dtypes

In [ ]:
df["FECHA_INICIO_TAO"] = pd.to_datetime(df["FECHA_INICIO_TAO"],format="%d/%m/%Y")
df["FECHA_NACIMIENTO"] = pd.to_datetime(df["FECHA_NACIMIENTO"],format="%d/%m/%Y")

In [ ]:
df["EDAD"] = round((df["FECHA_INICIO_TAO"] - df["FECHA_NACIMIENTO"]).dt.days/365.25,0)

In [ ]:
df["RATIO_NEUTROFILO/LINFOCITO"] = round(df["NEUTROFILOS"]/df["LINFOCITOS"],2)

df["ANEMIA"] = 0
df.loc[(df["SEXO"] == "H") & (df["HEMOGLOBINA"] < 13),"ANEMIA"] = 1
df.loc[(df["SEXO"] == "M") & (df["HEMOGLOBINA"] < 12),"ANEMIA"] = 1

df["TROMBOCITOPENIA"] = 0
df.loc[df["PLAQUETAS"] < 100,"TROMBOCITOPENIA"] = 1

df["CHA2DS2-VASC"] = 0
df.loc[df["EDAD"] >= 75,"CHA2DS2-VASC"] += 2
df.loc[(df["EDAD"] >= 65) & (df["EDAD"] < 75),"CHA2DS2-VASC"] += 1
df.loc[df["HTA"] == 1,"CHA2DS2-VASC"] += 1
df.loc[(df["ICTUS_PREVIO"] == 1) | (df["EMBOLIA_SISTEMICA_PREVIA"] == 1),"CHA2DS2-VASC"] += 2
df.loc[df["DM"] == 1,"CHA2DS2-VASC"] += 1
df.loc[(df["CARDIOPATIA_ISQUEMICA"] == 1) | (df["ARTERIOPATIA_PERIFERICA"] == 1),"CHA2DS2-VASC"] += 1
df.loc[df["SEXO"] == "M","CHA2DS2-VASC"] += 1
df.loc[df["INSUFICIENCIA_CARDIACA"] == 1,"CHA2DS2-VASC"] += 1

df["SAME-TT2R2"] = 0
df.loc[df["SEXO"] == "M","SAME-TT2R2"] += 1
df.loc[df["EDAD"] < 60,"SAME-TT2R2"] += 1
df.loc[(df["HTA"] + df["DM"] + df["CARDIOPATIA_ISQUEMICA"] + df["ARTERIOPATIA_PERIFERICA"] + df["INSUFICIENCIA_CARDIACA"] + df["ICTUS_PREVIO"] + df["ENFERMEDAD_PULMONAR"] + (df["ALTERACION_HEPATICA"] | df["INSUFICIENCIA_RENAL"]))>2,"SAME-TT2R2"] += 1
df.loc[(df["AMIODARONA"] == 1), "SAME-TT2R2"] += 1
df.loc[df["FUMA"]==1,"SAME-TT2R2"] += 2

df.drop(["FECHA_INICIO_TAO","FECHA_NACIMIENTO"],inplace=True,axis=1)

df["CUALQUIER_ANTIARRITMICO"] = 0
df.loc[(df["AMIODARONA"] == 1) | (df["OTROS_ANTIARRITMICOS"] == 1),"CUALQUIER_ANTIARRITMICO"] = 1

df["CONTROL_FRECUENCIA"] = 0
df.loc[(df["BETABLOQUEANTES"] == 1) | (df["SOTALOL"] == 1) | (df["VERAPAMILO"] == 1) | (df["DILTIAZEM"] == 1) | (df["DIGOXINA"] == 1),"CONTROL_FRECUENCIA"] = 1

df["CUALQUIER_ANTIAGREGANTE"] = 0
df.loc[(df["ASPIRINA"] == 1) | (df["OTROS_ANTIAGREGANTES"] == 1),"CUALQUIER_ANTIAGREGANTE"] = 1

df["DOBLE_ANTIAGREGACION"] = 0
df.loc[(df["ASPIRINA"] == 1) & (df["OTROS_ANTIAGREGANTES"] == 1),"DOBLE_ANTIAGREGACION"] = 1

df["EDAD_60"] = 1
df.loc[df["EDAD"] >= 60,"EDAD_60"] = 0

df["EDAD_65"] = df["EDAD"].apply(lambda x: 1 if x >= 65 else 0)

df["N_COMEDICACIONES"]= 0
df.loc[(df["CONTROL_FRECUENCIA"] == 1) | (df["CUALQUIER_ANTIARRITMICO"] == 1) ,"N_COMEDICACIONES"] = 1
df.loc[(df["CONTROL_FRECUENCIA"] == 1) & (df["CUALQUIER_ANTIARRITMICO"] == 1) ,"N_COMEDICACIONES"] = 2

df["ICTUS/EMBOLIA"] = 0
df.loc[(df["ICTUS_PREVIO"] == 1) | (df["EMBOLIA_SISTEMICA_PREVIA"] == 1),"ICTUS/EMBOLIA"] = 1

df["ENFERMEDAD_VASCULAR"] = 0
df.loc[(df["CARDIOPATIA_ISQUEMICA"] == 1) | (df["ARTERIOPATIA_PERIFERICA"] == 1),"ENFERMEDAD_VASCULAR"] = 1

df["DISFUNCION/DILATACION"] = 0
df.loc[(df["DISFUNCION_VENTRICULAR_EN_ECO"] == 1) | (df["TAMANO_AURICULA_IZQUIERDA"] == 1),"DISFUNCION/DILATACION"] = 1

df["SAME_MAS_DE_2"] = 0
df.loc[(df["HTA"] + df["DM"] + df["CARDIOPATIA_ISQUEMICA"]+ df["ARTERIOPATIA_PERIFERICA"]  + df["INSUFICIENCIA_CARDIACA"] + df["ICTUS_PREVIO"] + (df["ALTERACION_HEPATICA"] | df["INSUFICIENCIA_RENAL"]) + df["ENFERMEDAD_PULMONAR"] )>2,"SAME_MAS_DE_2"] = 1

df["ALTERACION"] = 0
df.loc[(df["ALTERACION_HEPATICA"] == 1) | (df["INSUFICIENCIA_RENAL"] == 1),"ALTERACION"] = 1

df["BLOQUEO_CALCIO"]=0
df.loc[(df["VERAPAMILO"]==1) | (df["DILTIAZEM"]==1),"BLOQUEO_CALCIO"]=1

df['SAME-TT2R2_MAS3'] = 0
df.loc[df['SAME-TT2R2'] > 2, 'SAME-TT2R2_MAS3'] = 1

dummies_neoplasia = pd.get_dummies(df['NEOPLASIA'], prefix='NEOPLASIA')
df = pd.concat([df, dummies_neoplasia], axis=1)
df.drop(["NEOPLASIA"],inplace=True,axis=1)

In [ ]:
# mirar que la suma de neutrofilos, linfocitos y monocitos es igual a la suma de los leucocitos (o -1)
df["SUMA_LEUCOCITOS"] = df["NEUTROFILOS"] + df["LINFOCITOS"] + df["MONOCITOS"]
df["DIFERENCIA_LEUCOCITOS"] = df["SUMA_LEUCOCITOS"] - df["LEUCOCITOS"]

# muestrame aquellos valores absolutos de la columna "DIFERENCIA_LEUCOCITOS" que sean mayores que 1
df.loc[abs(df["DIFERENCIA_LEUCOCITOS"]) > 1,"DIFERENCIA_LEUCOCITOS"]

In [ ]:
df.drop(["SUMA_LEUCOCITOS","DIFERENCIA_LEUCOCITOS"],inplace=True,axis=1)

In [ ]:
# rename columns
df = df.rename(columns={'NEOPLASIA_0': 'NEOPLASIA_NO'})

In [ ]:
# leer df_final
df = df[["ADE","MONOCITOS","NEUTROFILOS","EDAD","N_PASTILLAS_D_INICIO","DISLIPEMIA","AMIODARONA","INSUFICIENCIA_CARDIACA","ARTERIOPATIA_PERIFERICA","SAME_MAS_DE_2","SAME-TT2R2_MAS3","NEOPLASIA_NO","ENFERMEDAD_VASCULAR","TRT"]]

In [ ]:
df.head()

In [ ]:
# aplicar el logaritmo a las variables ADE, Monocitos y Neutrofilos
df["ADE"] = df["ADE"].apply(lambda x: np.log(x+1))
df["MONOCITOS"] = df["MONOCITOS"].apply(lambda x: np.log(x+1))
df["NEUTROFILOS"] = df["NEUTROFILOS"].apply(lambda x: np.log(x+1))

In [ ]:
# cambiar los valores mayores a 2.97 por 2.97 en la variable ADE
df.loc[df["ADE"] > 2.97,"ADE"] = 2.97
df.loc[df["ADE"] < 2.48,"ADE"] = 2.48

In [ ]:
df.head()

In [ ]:
# guardar el dataframe en un csv
df.to_csv("df_val.csv",index=False)

In [ ]:
# estandarizar

In [ ]:
# importra df_val
import pandas as pd
df_val_final = pd.read_csv("df_val_final.csv")

In [ ]:
df_val_final.head()

In [ ]:
df_val_final = df_val_final[["N_PASTILLAS_D_INICIO","ADE","MONOCITOS","DISLIPEMIA","SAME-TT2R2_MAS3","NEUTROFILOS","TRT"]]

In [ ]:
# separar df_val_final en X e y
X_val = df_val_final.drop(["TRT"],axis=1)
y_val = df_val_final["TRT"]

### MLPRegressor

In [ ]:
# importar MLPRegressor.pkl
import pickle
with open('MLPRegressor.pkl', 'rb') as file:
    mlp = pickle.load(file)


In [ ]:
# sacar las predicciones
import numpy as np

y_pred = mlp.predict(X_val)

# sacar metricas
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# mejores parametros
print("Mejores parametros: ", mlp.best_params_)

print("RMSE: ", np.sqrt(mean_squared_error(y_val, y_pred)))
print("MAE: ", mean_absolute_error(y_val, y_pred))
# std de MAE
print("std MAE: ", np.std(abs(y_val - y_pred)))
print("MSE: ", mean_squared_error(y_val, y_pred))
print("R2: ", r2_score(y_val, y_pred))
# r2 ajustado
print("R2 ajustado: ", 1 - (1-r2_score(y_val, y_pred))*(len(y_val)-1)/(len(y_val)-X_val.shape[1]-1))

In [ ]:
# sm
import statsmodels.api as sm
# plt
import matplotlib.pyplot as plt
# sns
import seaborn as sns

# residuos
residuos = y_val - y_pred

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(9, 8))

axes[0, 0].scatter(y_val, y_pred, edgecolors=(0, 0, 0), alpha = 0.4)
axes[0, 0].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()],
                'k--', color = 'black', lw=2)
axes[0, 0].set_title('Valor predicho vs valor real', fontsize = 10, fontweight = "bold")
axes[0, 0].set_xlabel('Real')
axes[0, 0].set_ylabel('Predicción')
axes[0, 0].tick_params(labelsize = 7)

axes[0, 1].scatter(list(range(len(y_val))), residuos,
                   edgecolors=(0, 0, 0), alpha = 0.4)
axes[0, 1].axhline(y = 0, linestyle = '--', color = 'black', lw=2)
axes[0, 1].set_title('Residuos del modelo', fontsize = 10, fontweight = "bold")
axes[0, 1].set_xlabel('id')
axes[0, 1].set_ylabel('Residuo')
axes[0, 1].tick_params(labelsize = 7)

sns.histplot(
    data    = residuos,
    stat    = "density",
    kde     = True,
    line_kws= {'linewidth': 1},
    color   = "firebrick",
    alpha   = 0.3,
    ax      = axes[1, 0]
)

axes[1, 0].set_title('Distribución residuos del modelo', fontsize = 10,
                     fontweight = "bold")
axes[1, 0].set_xlabel("Residuo")
axes[1, 0].tick_params(labelsize = 7)


sm.qqplot(
    residuos,
    fit   = True,
    line  = 'q',
    ax    = axes[1, 1], 
    color = 'firebrick',
    alpha = 0.4,
    lw    = 2
)
axes[1, 1].set_title('Q-Q residuos del modelo', fontsize = 10, fontweight = "bold")
axes[1, 1].tick_params(labelsize = 7)

axes[2, 0].scatter(y_pred, residuos,
                   edgecolors=(0, 0, 0), alpha = 0.4)
axes[2, 0].axhline(y = 0, linestyle = '--', color = 'black', lw=2)
axes[2, 0].set_title('Residuos del modelo vs predicción', fontsize = 10, fontweight = "bold")
axes[2, 0].set_xlabel('Predicción')
axes[2, 0].set_ylabel('Residuo')
axes[2, 0].tick_params(labelsize = 7)

# Se eliminan los axes vacíos
fig.delaxes(axes[2,1])

fig.tight_layout()
plt.subplots_adjust(top=0.9)
fig.suptitle('Diagnóstico residuos MLP', fontsize = 12, fontweight = "bold");

In [ ]:
# analizar la normalidad y homogeneidad de los residuos
from scipy.stats import shapiro
from scipy.stats import levene

# normalidad
stat, p = shapiro(y_val - y_pred)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')

### Regresion lineal

In [ ]:
# importar linear_regression.pkl
import pickle
with open('linear_regression.pickle', 'rb') as file:
    linear_regression = pickle.load(file)

In [ ]:
y_pred = linear_regression.predict(X_val)

# sacar metricas
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

print("RMSE: ", np.sqrt(mean_squared_error(y_val, y_pred)))
print("MAE: ", mean_absolute_error(y_val, y_pred))
# std de MAE
print("std MAE: ", np.std(abs(y_val - y_pred)))
print("MSE: ", mean_squared_error(y_val, y_pred))
print("R2: ", r2_score(y_val, y_pred))
# r2 ajustado
print("R2 ajustado: ", 1 - (1-r2_score(y_val, y_pred))*(len(y_val)-1)/(len(y_val)-X_val.shape[1]-1))

In [ ]:
# sm
import statsmodels.api as sm
# plt
import matplotlib.pyplot as plt
# sns
import seaborn as sns

# residuos
residuos = y_val - y_pred

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(9, 8))

axes[0, 0].scatter(y_val, y_pred, edgecolors=(0, 0, 0), alpha = 0.4)
axes[0, 0].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()],
                'k--', color = 'black', lw=2)
axes[0, 0].set_title('Valor predicho vs valor real', fontsize = 10, fontweight = "bold")
axes[0, 0].set_xlabel('Real')
axes[0, 0].set_ylabel('Predicción')
axes[0, 0].tick_params(labelsize = 7)

axes[0, 1].scatter(list(range(len(y_val))), residuos,
                   edgecolors=(0, 0, 0), alpha = 0.4)
axes[0, 1].axhline(y = 0, linestyle = '--', color = 'black', lw=2)
axes[0, 1].set_title('Residuos del modelo', fontsize = 10, fontweight = "bold")
axes[0, 1].set_xlabel('id')
axes[0, 1].set_ylabel('Residuo')
axes[0, 1].tick_params(labelsize = 7)

sns.histplot(
    data    = residuos,
    stat    = "density",
    kde     = True,
    line_kws= {'linewidth': 1},
    color   = "firebrick",
    alpha   = 0.3,
    ax      = axes[1, 0]
)

axes[1, 0].set_title('Distribución residuos del modelo', fontsize = 10,
                     fontweight = "bold")
axes[1, 0].set_xlabel("Residuo")
axes[1, 0].tick_params(labelsize = 7)


sm.qqplot(
    residuos,
    fit   = True,
    line  = 'q',
    ax    = axes[1, 1], 
    color = 'firebrick',
    alpha = 0.4,
    lw    = 2
)
axes[1, 1].set_title('Q-Q residuos del modelo', fontsize = 10, fontweight = "bold")
axes[1, 1].tick_params(labelsize = 7)

axes[2, 0].scatter(y_pred, residuos,
                   edgecolors=(0, 0, 0), alpha = 0.4)
axes[2, 0].axhline(y = 0, linestyle = '--', color = 'black', lw=2)
axes[2, 0].set_title('Residuos del modelo vs predicción', fontsize = 10, fontweight = "bold")
axes[2, 0].set_xlabel('Predicción')
axes[2, 0].set_ylabel('Residuo')
axes[2, 0].tick_params(labelsize = 7)

# Se eliminan los axes vacíos
fig.delaxes(axes[2,1])

fig.tight_layout()
plt.subplots_adjust(top=0.9)
fig.suptitle('Diagnóstico residuos Regresion lineal', fontsize = 12, fontweight = "bold");

In [ ]:
# analizar la normalidad y homogeneidad de los residuos
from scipy.stats import shapiro
from scipy.stats import levene

# normalidad
stat, p = shapiro(y_val - y_pred)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')

### Regresion logistica

In [ ]:
df_val_final = df_val_final[["ARTERIOPATIA_PERIFERICA","ENFERMEDAD_VASCULAR","SAME_MAS_DE_2","N_PASTILLAS_D_INICIO","NEOPLASIA_NO","ADE","MONOCITOS","DISLIPEMIA","AMIODARONA","EDAD","INSUFICIENCIA_CARDIACA","SAME-TT2R2_MAS3","NEUTROFILOS","TRT"]]

X_val = df_val_final.drop(["TRT"],axis=1)
y_val = df_val_final["TRT"]

In [ ]:
# importar logistic_regression.pkl
import pickle
with open('logistic_regression.pkl', 'rb') as file:
    logistic_regression = pickle.load(file)

In [ ]:
# metricas
from sklearn.metrics import f1_score, precision_score, recall_score, balanced_accuracy_score, roc_auc_score

# sacar las predicciones
y_pred = logistic_regression.predict(X_val)

# best params
print("Mejores parametros: ", logistic_regression.best_params_)
print("F1: ", f1_score(y_val, y_pred))
print("Precision: ", precision_score(y_val, y_pred))
print("Recall: ", recall_score(y_val, y_pred))
# balanced accuracy
print("Balanced accuracy: ", balanced_accuracy_score(y_val, y_pred))
# roc auc
print("ROC AUC: ", roc_auc_score(y_val, y_pred))

In [ ]:
# graficar la matriz de confusion
from sklearn.metrics import confusion_matrix
import seaborn as sns

matriz_confusion = confusion_matrix(y_val, y_pred)

sns.heatmap(matriz_confusion, annot=True, fmt='g', cmap='Blues')
plt.xlabel('Predicción')
plt.ylabel('Valor real')
plt.title('Matriz de confusión Regresión logística');

In [ ]:
# classification report
from sklearn.metrics import classification_report

print(classification_report(y_val, y_pred))

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

y_pred_proba = logistic_regression.predict_proba(X_val)[::,1]

fpr, tpr, _ = roc_curve(y_val,  y_pred_proba)
auc = roc_auc_score(y_val, y_pred_proba)
print(auc)
plt.plot(fpr,tpr,label="AUC="+str(round(auc,2)))
# axes
plt.ylabel("Ratio Verdaderos Positivos")
plt.xlabel("Ratio Falsos Positivos")
# linea diagonal en gris claro
plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
# titulo
plt.title("Curva ROC regresión logística")
plt.legend(loc=4)
plt.show()

### KNN

In [ ]:
# importar KNeighborsClassifier.pkl
import pickle
with open('KNeighborsClassifier60.pickle', 'rb') as file:
    KNeighborsClassifier = pickle.load(file)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, balanced_accuracy_score, roc_auc_score

# sacar las predicciones
y_pred = KNeighborsClassifier.predict(X_val)

# best params
print("Mejores parametros: ", KNeighborsClassifier.best_params_)
print("F1: ", f1_score(y_val, y_pred))
print("Precision: ", precision_score(y_val, y_pred))
print("Recall: ", recall_score(y_val, y_pred))
# balanced accuracy
print("Balanced accuracy: ", balanced_accuracy_score(y_val, y_pred))
# roc auc
print("ROC AUC: ", roc_auc_score(y_val, y_pred))

In [ ]:
# matriz de confusion
from sklearn.metrics import confusion_matrix
import seaborn as sns

matriz_confusion = confusion_matrix(y_val, y_pred)

sns.heatmap(matriz_confusion, annot=True, fmt='g', cmap='Blues')
plt.xlabel('Predicción')
plt.ylabel('Valor real')
plt.title('Matriz de confusión KNN');

In [ ]:
# classification report
from sklearn.metrics import classification_report

print(classification_report(y_val, y_pred))

In [ ]:
# curva roc
from sklearn.metrics import roc_curve

y_pred_proba = KNeighborsClassifier.predict_proba(X_val)[::,1]   

fpr, tpr, _ = roc_curve(y_val,  y_pred_proba)
auc = roc_auc_score(y_val, y_pred_proba)
print(auc)
plt.plot(fpr,tpr,label="AUC="+str(round(auc,2)))
# axes
plt.ylabel("Ratio Verdaderos Positivos")
plt.xlabel("Ratio Falsos Positivos")
# linea diagonal en gris claro
plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
# titulo
plt.title("Curva ROC KNN")
plt.legend(loc=4)
plt.show()